## Домашняя работа №6<br>
<br>
Импортируем необходимые пакеты.

In [1]:
import sys 
import modeller 
import _modeller
import modeller.automodel
import nglview

Скачаем pdb лизоцима из радужной форели и последовательность лизоцима из виргинской американской куропатки.

In [2]:
%%bash
wget http://www.pdb.org/pdb/files/1lmp.pdb
wget http://www.uniprot.org/uniprot/P00700.fasta

--2017-12-09 15:36:33--  http://www.pdb.org/pdb/files/1lmp.pdb
Resolving www.pdb.org (www.pdb.org)... 128.6.244.52
Connecting to www.pdb.org (www.pdb.org)|128.6.244.52|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.rcsb.org/pdb/files/1lmp.pdb [following]
--2017-12-09 15:36:34--  https://www.rcsb.org/pdb/files/1lmp.pdb
Resolving www.rcsb.org (www.rcsb.org)... 132.249.213.110
Connecting to www.rcsb.org (www.rcsb.org)|132.249.213.110|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://files.rcsb.org/view/1lmp.pdb [following]
--2017-12-09 15:36:34--  http://files.rcsb.org/view/1lmp.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.140
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: `1lmp.pdb'

     0K .......... .......... .......... .......... ..........  11

In [5]:
env=modeller.environ()
env.io.hetatm=True

# Создаем выравнивание
alignm=modeller.alignment(env)
alignm.append(file='P00700.fasta', align_codes='all',alignment_format='FASTA')
mdl = modeller.model(env, file='1lmp.pdb', model_segment=('FIRST:'+'A', 'LAST:'+'A'))
alignm.append_model(mdl, atom_files='1lmp.pdb', align_codes='1lmp')
alignm[0].code = 'model'
alignm[1].code = 'reference'

# Производим выравнивание
alignm.salign()
alignm.write(file='all_in_one.ali', alignment_format='PIR')

# Выбираем объект для моделирования 
s = alignm[0]
pdb = alignm[1]

# Создаем объект automodel
a = modeller.automodel.automodel(env, alnfile='all_in_one.ali', knowns = pdb.code, sequence = s.code)
a.name='mod'+s.code
a.starting_model = 1
a.ending_model = 1
a.make()

read_pd_459W> Residue type  NAG not recognized. 'automodel' model building
              will treat this residue as a rigid body.
              To use real parameters, add the residue type to ${LIB}/restyp.lib,
              its topology to ${LIB}/top_*.lib, and suitable forcefield
              parameters to ${LIB}/par.lib.
rdpdb___459W> Residue type  NDG not recognized. 'automodel' model building
              will treat this residue as a rigid body.
              To use real parameters, add the residue type to ${LIB}/restyp.lib,
              its topology to ${LIB}/top_*.lib, and suitable forcefield
              parameters to ${LIB}/par.lib.

SALIGN_____> adding the next group to the alignment; iteration    1
fndatmi_285W> Only      129 residues out of      132 contain atoms of type  CA
              (This is usually caused by non-standard residues, such
              as ligands, or by PDB files with missing atoms.)
fndatmi_285W> Only      129 residues out of      132 contain atoms

Посмотрим, что получилось.

In [6]:
nglview.show_structure_file('model.B99990001.pdb')

NGLWidget()

Добавим лиганд

In [7]:
stri = ''
for i in alignm[0].residues:
    stri += i.code
stri += '...'
alignm.append_sequence(stri)

# Выбираем объект для моделирования 
s = alignm[2]
pdb = alignm[1]
s.code = 'model_ligand'

alignm.salign()
alignm.write(file='all_in_one.ali', alignment_format='PIR')

# Создаем объект automodel
a = modeller.automodel.automodel(env, alnfile='all_in_one.ali', knowns= pdb.code , sequence = s.code )
a.name='mod'+s.code
a.starting_model = 1
a.ending_model = 1
a.make()


SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    2
automodel__W> Topology and/or parameter libraries already in memory. These will
                be used instead of the automodel defaults. If this is not what you
                want, clear them before creating the automodel object with
                env.libs.topology.clear() and env.libs.parameters.clear()
fndatmi_285W> Only      129 residues out of      132 contain atoms of type  CA
              (This is usually caused by non-standard residues, such
              as ligands, or by PDB files with missing atoms.)
fndatmi_285W> Only      129 residues out of      132 contain atoms of type  CA
              (This is usually caused by non-standard residues, such
              as ligands, or by PDB files with missing atoms.)

check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 a

Белок с лигандом:

In [9]:
nglview.show_structure_file('model_ligand.B99990001.pdb')

NGLWidget()